In [1]:
"""Before starting, check that vanilla and JIT-compiled functions give the same results...
The proper test code is already part of the library file, so just run it.
"""
%run secondlayer.py

Performing some poor man's (not so unit) testing.

Testing 'gdummyv' implementations: 	OK!
Testing 'gdummynv' implementations: 	OK!
Testing 'gdummynvana' implementations: 	OK!

If you are reading this, then everything seems to have run nicely :).


In [2]:
import numpy as np
from secondlayer import (_gdummyv_vanilla, _gdummyv_numpy, _gdummyv_jit,
                         _gdummynv_vanilla, _gdummynv_numpy, _gdummynv_jit,
                         _gdummynvana_vanilla, _gdummynvana_numpy, _gdummynvana_jit)

# ##################
# Helper function(s)
def get_dummy_data(seed=None):
    _prng = np.random.RandomState(seed)  # for reproducibility
    m = 4*10  # number of features
    n = 3*10  # number classes
    gmin, gmax = 1e-8, 1e-6
    g = 256  # number of writable levels
    G = _prng.uniform(low=gmin, high=gmax, size=(2*m + 2, n))
    error = _prng.random_sample(size=G.shape)
    deltas = np.zeros(n)
    gmins = _prng.normal(loc=gmin, scale=0.2 * gmin, size=G.shape)
    gmaxs = _prng.normal(loc=gmax, scale=0.2 * gmax, size=G.shape)
    memarray = np.dstack((gmaxs, gmins))

    return G, gmax, gmin, g, m, n, error, deltas, memarray


In [3]:
# Run the JIT versions once to compile them

G0, _, _, g, m, n, error, _, memarray = get_dummy_data()
G0 = _gdummyv_jit(G0, g, m, n, error, memarray)

G1, gmax, gmin, g, m, n, error, _, _ = get_dummy_data()
G1 = _gdummynv_jit(G1, gmax, gmin, g, m, n, error)
              
G2, gmax, gmin, g, m, n, error, deltas, _ = get_dummy_data()
G2 = _gdummynvana_jit(G2, gmax, gmin, g, m, n, error, deltas)

In [4]:
print("### Benchmarking `gdummyv` implementations ###")
seed = np.random.randint(2134)

for func, case in ((_gdummyv_vanilla, "Vanilla:"),
                   (_gdummyv_numpy, "Numpy-only:"),
                   (_gdummyv_jit, "JIT-compiled:")):
    G0, _, _, g, m, n, error, _, memarray = get_dummy_data(seed)
    print(case, end="\t")
    %timeit func(G0, g, m, n, error, memarray)

### Benchmarking `gdummyv` implementations ###
Vanilla:	10 loops, best of 3: 19.2 ms per loop
Numpy-only:	10000 loops, best of 3: 103 µs per loop
JIT-compiled:	10000 loops, best of 3: 21.4 µs per loop


In [5]:
print("### Benchmarking `gdummynv` implementations ###")
seed = np.random.randint(3241)

for func, case in ((_gdummynv_vanilla, "Vanilla:"),
                   (_gdummynv_numpy, "Numpy-only:"),
                   (_gdummynv_jit, "JIT-compiled:")):
    G1, gmax, gmin, g, m, n, error, _, _ = get_dummy_data(seed)
    print(case, end="\t")
    %timeit func(G1, gmax, gmin, g, m, n, error)

### Benchmarking `gdummynv` implementations ###
Vanilla:	100 loops, best of 3: 8.1 ms per loop
Numpy-only:	10000 loops, best of 3: 53.6 µs per loop
JIT-compiled:	100000 loops, best of 3: 10.3 µs per loop


In [6]:
print("### Benchmarking `gdummynvana` implementations ###")
seed = np.random.randint(4231)

for func, case in ((_gdummynvana_vanilla, "Vanilla:"),
                   (_gdummynvana_numpy, "Numpy-only:"),
                   (_gdummynvana_jit, "JIT-compiled:")):
    G2, gmax, gmin, g, m, n, error, deltas, _ = get_dummy_data(seed)
    print(case, end="\t")
    %timeit func(G2, gmax, gmin, g, m, n, error, deltas)

# Remark: why is the JIT-compiled version so slow in this case?!

### Benchmarking `gdummynvana` implementations ###
Vanilla:	100 loops, best of 3: 9.36 ms per loop
Numpy-only:	The slowest run took 4.14 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 87.6 µs per loop
JIT-compiled:	100 loops, best of 3: 3.91 ms per loop
